**정확도를 산출한다**

In [4]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


**전처리는 동일하다**

In [5]:
import pandas as pd
import numpy as np

data = pd.read_csv("/gdrive/My Drive/classification/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
df_sample = data.loc[:, ['Init_Win_bytes_forward', ' Avg Fwd Segment Size',' Init_Win_bytes_backward', ' Fwd Header Length.1',' Label']]
df_labels= df_sample[' Label']
df_packets=df_sample.drop([' Label'],axis='columns')

#같은 길이 나오는 거확인
print(len(df_sample))
print(len(df_labels))
print(len(df_packets))

#콜롬 목록과 데이터 타입 확인 후 변환
# df_labels.columns #series라 안됨
df_packets.columns
#type(df_sample)
df_packets = df_packets.astype(float) #형변환

#리스트로 되돌리기
packets= df_packets.values.tolist()
labels= df_labels.values.tolist()


#확인
packets[1]
labels[1]

#길이 확인
len(packets)
len(labels)
#len(packets[1]) #한개당 80차원

#기왕 하는거 labels도 float으로 바꿔주기
for c in range(225745):
  if (labels[c] == 'BENIGN' ):
    labels[c]=0 #false
  else:
    labels[c] =1 #true
 

#테스트 군과 학습군으로 나누기.
#전체는 225745개. (70%는 약 158,021 개)
train_set = packets[:158021]
test_set = packets[158021:]
train_label =labels[:158021]
test_label = labels[158021:]

train_set = np.asarray(train_set)
train_label = np.asarray(train_label)
test_set = np.asarray(test_set)
test_label = np.asarray(test_label)


#잘 나눠졌는지 확인
print(len(train_set) + len(test_set))
print(len(test_label) + len(train_label))


225745
225745
225745
225745
225745


In [0]:
#사용자 정의 함수
from keras import backend as K

def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())
    return _f1score

def recall(y_target, y_pred):
    y_target_yn = K.round(K.clip(y_target, 0, 1))
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) 
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 
    count_true_positive_false_negative = K.sum(y_target_yn)
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())
    return recall


def precision(y_target, y_pred):
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) 
    y_target_yn = K.round(K.clip(y_target, 0, 1)) 
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 
    count_true_positive_false_positive = K.sum(y_pred_yn)
    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())
    return precision



In [0]:
#모델 만들기
import numpy as np
import tensorflow as tf

model = tf.keras.Sequential([
  tf.keras.layers.Conv1D(16, (2), activation='relu', input_shape=(4, 1)),
  tf.keras.layers.MaxPooling1D(2,padding='same'),
  tf.keras.layers.Conv1D(16, (2), activation='relu'),
  tf.keras.layers.MaxPooling1D(2, padding='same'),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  #dropout
  #tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(256, activation='relu'),
  #dropout
  #tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  #dropout
  #tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(15, activation='softmax')
])
model.summary() #모델 출력

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_6 (Conv1D)            (None, 3, 16)             48        
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 2, 16)             0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 1, 16)             528       
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 1, 16)             0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 16)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 512)               8704      
_________________________________________________________________
dense_13 (Dense)             (None, 256)              

In [0]:
from sklearn.metrics import recall_score 
from sklearn.metrics import precision_score 
from sklearn.metrics import f1_score 

train_set = np.expand_dims(train_set, axis=2) #reshape
test_set = np.expand_dims(test_set, axis=2) #reshape

#학습.
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy', precision, recall, f1score])

history = model.fit(train_set, train_label, epochs=5,
                    validation_data=(test_set, test_label))
test_loss = model.evaluate(test_set, test_label)



Epoch 1/5
4939/4939 [==============================] - 24s 5ms/step - loss: 0.5546 - accuracy: 0.9705 - precision: 0.6406 - recall: 1.0000 - f1score: 0.7776 - val_loss: 0.1095 - val_accuracy: 0.9877 - val_precision: 0.3961 - val_recall: 0.3987 - val_f1score: 0.3966
Epoch 2/5
4939/4939 [==============================] - 24s 5ms/step - loss: 0.0629 - accuracy: 0.9867 - precision: 0.6403 - recall: 1.0000 - f1score: 0.7775 - val_loss: 0.0643 - val_accuracy: 0.9877 - val_precision: 0.3961 - val_recall: 0.3987 - val_f1score: 0.3966
Epoch 3/5
4939/4939 [==============================] - 24s 5ms/step - loss: 0.0440 - accuracy: 0.9926 - precision: 0.6403 - recall: 1.0000 - f1score: 0.7774 - val_loss: 0.1107 - val_accuracy: 0.9890 - val_precision: 0.3961 - val_recall: 0.3987 - val_f1score: 0.3966
Epoch 4/5
4939/4939 [==============================] - 24s 5ms/step - loss: 0.0285 - accuracy: 0.9947 - precision: 0.6403 - recall: 1.0000 - f1score: 0.7774 - val_loss: 0.0759 - val_accuracy: 0.9878 - v

**공격 정상 여부 확인**

In [0]:
#BENIGN
test=np.asarray(packets[0])
print('classification result+', model.predict_classes(test.reshape(1,4,1)))

#DDOS
test2=np.asarray(packets[18886])
print('classification result+', model.predict_classes(test2.reshape(1,4,1)))

classification result+ [0]
classification result+ [1]


In [7]:
#답이 맞는지 체크
print(labels[0])
print(labels[18886])

0
1
